In [3]:
from __future__ import division
import pickle
import os
import types
import random
import uuid
import math
from copy import deepcopy as copy
import logging

import gym
from gym import spaces
from gym.envs.classic_control import rendering
import numpy as np
import tensorflow as tf
from scipy.misc import logsumexp
from baselines import deepq
import baselines.common.tf_util as U




In [4]:
from nn_dynamics import MPCController

ModuleNotFoundError: No module named 'rllab'

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=False)

In [ ]:
# logger = logging.getLogger('root')
# print(logger.hasHandlers())
# assert len(logger.handlers) == 1
# handler = logger.handlers[0]
# handler.setLevel(logging.WARNING)

In [ ]:
data_dir = os.path.join('data', '4.0')

In [ ]:
sess = tf.Session()

create envs, pilot policies

In [ ]:
throttle_mag = 0.75
def disc_to_cont(action):
  if type(action) == np.ndarray:
    return action
  # main engine
  if action < 3:
    m = -throttle_mag
  elif action < 6:
    m = throttle_mag
  else:
    raise ValueError
  # steering
  if action % 3 == 0:
    s = -throttle_mag
  elif action % 3 == 1:
    s = 0
  else:
    s = throttle_mag
  return np.array([m, s])

In [ ]:
n_act_dim = 6
n_obs_dim = 9

In [ ]:
max_ep_len = 1000

In [ ]:
fps = 40

grid search

In [ ]:
train_goals = np.arange(1, 10, 1).astype(int)
n_train_tasks = train_goals.size

In [ ]:
def make_lander_env(fps=fps, goal=None):
  env = gym.make('LunarLanderContinuous-v2')
  env.unwrapped.goal = goal
  env.action_space = spaces.Discrete(n_act_dim)
  env.unwrapped._step_orig = env.unwrapped.step
  def step(self, action):
    obs, r, done, info = self._step_orig(disc_to_cont(action))
    return obs, r, done, info
  env.unwrapped.step = types.MethodType(step, env.unwrapped)
  env.unwrapped.fps = fps
  return env

In [ ]:
# train_newton_envs = [make_lander_env(fps=fast_fps, goal=goal) for goal in train_goals]
train_envs = [make_lander_env(fps=fps, goal=goal) for goal in train_goals]

In [ ]:
def run_ep(policy, env, max_ep_len=max_ep_len, render=False, task_idx=None):
  obs = env.reset()
  done = False
  totalr = 0.
  prev_obs = obs
  rollout = []
  for step_idx in range(max_ep_len+1):
    if done:
      break
    action = policy(obs)
    obs, r, done, info = env.step(action)
    rollout.append((prev_obs, action, r, obs, float(done), task_idx))
    prev_obs = obs
    if render:
      env.render()
    totalr += r
  return rollout

train agent with soft dqn

In [ ]:
train_env = make_lander_env(fps=fps)

In [ ]:
n_training_episodes = 800
load_pretrained_pilot = True

In [ ]:
make_q_func = lambda: deepq.models.mlp([64, 64])
dqn_learn_kwargs = {
  'lr': 1e-3,
  'target_network_update_freq': 3000,
  'print_freq': 100,
  'max_timesteps': max_ep_len * (1 if load_pretrained_pilot else n_training_episodes)
}

In [ ]:
with open(os.path.join(data_dir, 'dqn_pilot_scope.pkl'), 'rb') as f:
  dqn_pilot_scope = pickle.load(f)

In [ ]:
# dqn_pilot_scope = str(uuid.uuid4())

In [ ]:
train_env.action_space #.contains(np.int64(1))

In [ ]:
raw_dqn_pilot_policy, _ = deepq.learn(
  train_env,
  q_func=make_q_func(),
  scope=dqn_pilot_scope,
  **dqn_learn_kwargs
)

In [ ]:
# with open(os.path.join(data_dir, 'dqn_pilot_scope.pkl'), 'wb') as f:
#   pickle.dump(dqn_pilot_scope, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
dqn_pilot_path = os.path.join(data_dir, 'dqn_pilot.tf')

In [ ]:
def save_tf_vars(sess, scope, path):
  saver = tf.train.Saver([v for v in tf.global_variables() if v.name.startswith(scope + '/')])
  saver.save(sess, save_path=path)

In [ ]:
def load_tf_vars(sess, scope, path):
  saver = tf.train.Saver([v for v in tf.global_variables() if v.name.startswith(scope + '/')])
  saver.restore(sess, path)

In [ ]:
# save_tf_vars(U.get_session(), dqn_pilot_scope, dqn_pilot_path)

In [ ]:
load_tf_vars(U.get_session(), dqn_pilot_scope, dqn_pilot_path)

In [ ]:
VIEWPORT_W = 600
VIEWPORT_H = 400
SCALE = 30.0
W = VIEWPORT_W/SCALE
H = VIEWPORT_H/SCALE
CHUNKS = 11
chunk_x = [W/(CHUNKS-1)*i for i in range(CHUNKS)]
helipad_xs = [(chunk_x[goal-1]+chunk_x[goal+1])/2 for goal in train_goals]
train_goal_obses = [(helipad_x - VIEWPORT_W/SCALE/2) / (VIEWPORT_W/SCALE/2) for helipad_x in helipad_xs]

In [ ]:
temperature = 1
def make_pilot_policy(train_task_idx):
  goal_obs = train_goal_obses[train_task_idx]
  def pilot_policy(obs):
    my_obs = copy(obs)
    my_obs[8] = goal_obs
    with tf.variable_scope(dqn_pilot_scope, reuse=None):
      return raw_dqn_pilot_policy._act(my_obs[None, :], temperature=temperature)[0]
  return pilot_policy

In [ ]:
pilot_policies = [make_pilot_policy(train_task_idx) for train_task_idx in range(n_train_tasks)]

In [ ]:
def make_pilot_policy(train_task_idx):
  return pilot_policies[train_task_idx]

sanity-check envs, agents

In [ ]:
# train_task_idx = 0

In [ ]:
# run_ep(pilot_policies[train_task_idx], train_envs[train_task_idx], render=True)

In [ ]:
# train_envs[train_task_idx].close()

In [ ]:
# run_ep(pilot_policies[train_task_idx], train_newton_envs[train_task_idx], render=True)

In [ ]:
# train_newton_envs[train_task_idx].close()

fit internal dynamics model

In [ ]:
n_train_rollouts_per_env = 1000

In [ ]:
# demo_rollouts = [[run_ep(pilot_policies[train_task_idx], newton_env, render=False, task_idx=train_task_idx)          
#                   for _ in range(n_train_rollouts_per_env)]
#                  for train_task_idx, newton_env in enumerate(train_newton_envs)]

In [ ]:
# with open(os.path.join(data_dir, 'pilot_policy_demo_rollouts.pkl'), 'wb') as f:
#   pickle.dump(demo_rollouts, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# demo_rollouts_path = os.path.join('data', '5.1-lander-newton', 'sid_pilot_policy_demo_rollouts.pkl')
demo_rollouts_path = os.path.join(data_dir, 'pilot_policy_demo_rollouts.pkl')

In [ ]:
with open(demo_rollouts_path, 'rb') as f:
  demo_rollouts = pickle.load(f)

In [ ]:
def build_mlp(
    input_placeholder,
    output_size,
    scope,
    n_layers=1,
    size=256,
    activation=tf.nn.relu,
    output_activation=None,
    reuse=False
  ):
  out = input_placeholder
  with tf.variable_scope(scope, reuse=reuse):
    for _ in range(n_layers):
      out = tf.layers.dense(out, size, activation=activation)
    out = tf.layers.dense(out, output_size, activation=output_activation)
  return out

In [ ]:
n_obs_feats = n_obs_dim
featurize_obs = lambda s: s
n_act_feats = 2
featurize_act = lambda a: [disc_to_cont(act) for act in a]

In [ ]:
def vectorize_rollouts(rollouts):
  obs = [[] for _ in range(n_train_tasks)]
  actions = [[] for _ in range(n_train_tasks)]
  next_obs = [[] for _ in range(n_train_tasks)]
  for task_idx, task_rollouts in enumerate(rollouts):
    for task_rollout in task_rollouts:
      more_obs, more_actions, _, more_next_obs = list(zip(*task_rollout))[:4]
      obs[task_idx].extend([featurize_obs(s) for s in more_obs])
      actions[task_idx].extend(more_actions)
      next_obs[task_idx].extend([featurize_obs(s) for s in more_next_obs])
  l = min(len(x) for x in obs)
  idxes = [random.sample(list(range(len(x))), l) for x in obs]
  f = lambda x: np.array(x[1])[idxes[x[0]]]
  obs = np.array(list(map(f, enumerate(obs))))
  actions = np.array(list(map(f, enumerate(actions))))
  action_feats = np.array([featurize_act(a) for a in actions])
  next_obs = np.array(list(map(f, enumerate(next_obs))))

  return obs, actions, action_feats, next_obs

In [ ]:
demo_obs = None
demo_actions = None
demo_act_feats = None
demo_next_obs = None
demo_task_idxes = None
train_demo_example_idxes = None
val_demo_batch = None

In [ ]:
def process_demo_rollouts(demo_rollouts):
  global demo_obs
  global demo_actions
  global demo_act_feats
  global demo_next_obs
  global demo_task_idxes
  global train_demo_example_idxes
  global val_demo_batch

  
  demo_obs, demo_actions, demo_act_feats, demo_next_obs = vectorize_rollouts(demo_rollouts)
  demo_example_idxes = list(range(demo_obs.shape[1]))
  
  random.shuffle(demo_example_idxes)
  n_train_demo_examples = int(0.9 * len(demo_example_idxes))
  train_demo_example_idxes = demo_example_idxes[:n_train_demo_examples]
  val_demo_example_idxes = demo_example_idxes[n_train_demo_examples:]
  val_demo_batch = (demo_obs[:, val_demo_example_idxes], demo_actions[:, val_demo_example_idxes],
        demo_act_feats[:, val_demo_example_idxes], demo_next_obs[:, val_demo_example_idxes])

In [ ]:
process_demo_rollouts(demo_rollouts)

In [ ]:
def sample_batch(size):
  idxes = random.sample(train_demo_example_idxes, size)
  demo_batch = demo_obs[:, idxes], demo_actions[:, idxes], demo_act_feats[:, idxes], demo_next_obs[:, idxes]
  return demo_batch

In [ ]:
gamma = 0.99
iterations = 100000
learning_rate = 1e-3
batch_size = 512 // n_train_tasks
sq_td_err_penalty = 1e-3
trans_err_penalty = 1e0


q_n_layers = 1
q_layer_size = 32
q_activation = tf.nn.relu
q_output_activation = None

invdyn_n_layers = 1
invdyn_layer_size = 32
invdyn_activation = tf.nn.relu
invdyn_output_activation = None


constraint_sampling_freq = 100000
constraint_batch_size = batch_size
n_constraint_rollouts_per_env = 100

val_update_freq = 100

In [ ]:
im_scope = str(uuid.uuid4())
q_scope = str(uuid.uuid4())
invdyn_scope = str(uuid.uuid4())

In [ ]:
demo_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_feats], name="dot")
demo_act_t_ph = tf.placeholder(tf.int32, [n_train_tasks, None], name="dat")
demo_act_t_feats_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_act_feats], name="datf")
demo_next_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_feats], name="dnot")
demo_batch_size_ph = tf.placeholder(tf.int32, name="dbs")


constraint_obs_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_obs_feats], name="cot")
constraint_act_t_ph = tf.placeholder(tf.int32, [n_train_tasks, None], name="cat")
constraint_act_t_feats_ph = tf.placeholder(tf.float32, [n_train_tasks, None, n_act_feats], name="catf")
constraint_rew_t_ph = tf.placeholder(tf.float32, [n_train_tasks, None], name="crt")
constraint_batch_size_ph = tf.placeholder(tf.int32, name="cbs")

In [ ]:
demo_batch_idxes = tf.reshape(
  tf.range(0, demo_batch_size_ph, 1), 
  [demo_batch_size_ph, 1])

extract_task = lambda x, i: tf.squeeze(tf.gather(x, tf.convert_to_tensor(
  [i], dtype=tf.int32)), axis=[0]) 

demo_q_t = tf.stack([tf.gather_nd(
  build_mlp(
    extract_task(demo_obs_t_ph, train_task_idx),
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation
  ), 
  tf.concat([
    demo_batch_idxes, 
    tf.expand_dims(extract_task(demo_act_t_ph, train_task_idx), 1)], axis=1)
) for train_task_idx in range(n_train_tasks)], axis=0)

demo_v_t = tf.reduce_logsumexp(
  tf.stack([build_mlp(
    extract_task(demo_obs_t_ph, train_task_idx),
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation,
    reuse=True
  ) for train_task_idx in range(n_train_tasks)], axis=0),
  axis=2)

act_log_likelihoods = demo_q_t - demo_v_t

In [ ]:
neg_avg_log_likelihood = -tf.reduce_mean(act_log_likelihoods)

In [ ]:
pred_next_obs = build_mlp(
  tf.concat((demo_obs_t_ph, demo_act_t_feats_ph), axis=2),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation)

In [ ]:
constraint_act_t_feats_reshaped = tf.reshape(
  constraint_act_t_feats_ph, [n_train_tasks*constraint_batch_size_ph, n_act_dim])

constraint_obs_t_reshaped = tf.reshape(
  constraint_obs_t_ph, [n_train_tasks*constraint_batch_size_ph, n_obs_feats])

demo_act_t_feats_reshaped = tf.reshape(
  demo_act_t_feats_ph, [n_train_tasks*demo_batch_size_ph, n_act_feats])

demo_obs_t_reshaped = tf.reshape(
  demo_obs_t_ph, [n_train_tasks*demo_batch_size_ph, n_obs_feats])

demo_next_obs_t_reshaped = tf.reshape(
  demo_next_obs_t_ph, [n_train_tasks, demo_batch_size_ph, n_obs_feats])

demo_pred_next_obs_t_reshaped = tf.reshape(
    pred_next_obs, [n_train_tasks, demo_batch_size_ph, n_obs_feats])

In [ ]:
trans_err = demo_pred_next_obs_t_reshaped - demo_next_obs_t_reshaped

In [ ]:
trans_loss = tf.reduce_mean(trans_err**2)

In [ ]:
# Predicted constraint next state given inv dyns
constraint_obs_tp1 = tf.reshape(build_mlp(
  tf.concat((constraint_obs_t_ph, constraint_act_t_feats_ph), axis=2),
  n_obs_dim, invdyn_scope, 
  n_layers=invdyn_n_layers, size=invdyn_layer_size,
  activation=invdyn_activation, output_activation=invdyn_output_activation,
  reuse=True), [n_train_tasks, constraint_batch_size_ph, n_obs_feats])

In [ ]:
q_tp1 = tf.stack([build_mlp(
    extract_task(constraint_obs_tp1, train_task_idx), 
    n_act_dim, q_scope+'-'+str(train_task_idx),
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation, 
    reuse=True) for train_task_idx in range(n_train_tasks)], axis=0)
v_tp1 = tf.reduce_logsumexp(q_tp1, axis=2)

In [ ]:
target_t = constraint_rew_t_ph + gamma * v_tp1

In [ ]:
constraint_batch_idxes = tf.reshape(
  tf.range(0, constraint_batch_size_ph, 1), 
  [constraint_batch_size_ph, 1])

# Sampled constraint state q-vals
q_t = tf.stack([tf.gather_nd(
  build_mlp(
    extract_task(constraint_obs_t_ph, train_task_idx), 
    n_act_dim, q_scope+'-'+str(train_task_idx), 
    n_layers=q_n_layers, size=q_layer_size,
    activation=q_activation, output_activation=q_output_activation, 
    reuse=True
  ), 
  tf.concat([
    constraint_batch_idxes, 
    tf.expand_dims(extract_task(constraint_act_t_ph, train_task_idx), 1)], axis=1)
) for train_task_idx in range(n_train_tasks)], axis=0)

In [ ]:
# Bellman errors
td_err = q_t - target_t

In [ ]:
sq_td_err = tf.reduce_mean(td_err**2)

In [ ]:
loss = neg_avg_log_likelihood + sq_td_err_penalty * sq_td_err + trans_err_penalty * trans_loss

In [ ]:
full_update_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
trans_update_op = tf.train.AdamOptimizer(learning_rate).minimize(trans_loss)

In [ ]:
def sample_constraints(_):
  constraint_rollouts = [[] for _ in range(n_train_tasks)]
  
  for train_task_idx in range(n_train_tasks):
    rollouts = [[] for _ in range(n_constraint_rollouts_per_env)]
    envs = [make_lander_env(
      fps=fps, goal=train_goals[train_task_idx]) for _ in range(
      n_constraint_rollouts_per_env)]
    obses = np.array([env.reset() for env in envs])
    dones = [False for _ in envs]
    prev_obses = obses
    for step_idx in range(max_ep_len+1):
      not_done_idxes = [i for i, done in enumerate(dones) if not done]
      batch_size = len(not_done_idxes)
      if batch_size == 0:
        break
      actions = np.random.choice(n_act_dim, batch_size)
      for i, env_idx in enumerate(not_done_idxes):
        env = envs[env_idx]
        action = actions[i]
        obs, r, done, info = env.step(action)
        obses[env_idx] = obs
        dones[env_idx] = done
        rollouts[env_idx].append((
          prev_obses[env_idx], action, r))
      prev_obses = copy(obses)
    constraint_rollouts[train_task_idx].extend([r for r in rollouts if r != []])

  size = min(sum(len(r) for r in rollouts) for rollouts in constraint_rollouts)
  
  global train_constraint_example_idxes
  global val_constraint_batch
  global constraint_obs_t
  global constraint_act_t
  global constraint_rew_t
  global constraint_act_t_feats
    
  constraint_obs_t = np.zeros((n_train_tasks, size, n_obs_feats))
  constraint_act_t = np.zeros((n_train_tasks, size))
  constraint_act_t_feats = np.zeros((n_train_tasks, size, n_act_feats))
  constraint_rew_t = np.zeros((n_train_tasks, size))
  
  for train_task_idx in range(n_train_tasks):
    unfeat_obses, actions, rews = list(zip(*sum(
      constraint_rollouts[train_task_idx], [])))
    obses = [featurize_obs(s) for s in unfeat_obses]
    act_feats = [disc_to_cont(a) for a in actions]
    idxes = random.sample(list(range(len(obses))), size)
    constraint_obs_t[train_task_idx, :, :] = np.array(obses)[idxes, :]
    constraint_act_t[train_task_idx, :] = np.array(actions)[idxes]
    constraint_rew_t[train_task_idx, :] = np.array(rews)[idxes]
    constraint_act_t_feats[train_task_idx, :, :] = np.array(act_feats)[idxes, :]

  
  constraint_example_idxes = list(range(size))
  random.shuffle(constraint_example_idxes)
  n_train_constraint_examples = int(0.9 * size)
  
  train_constraint_example_idxes = constraint_example_idxes[:n_train_constraint_examples]
  val_constraint_example_idxes = constraint_example_idxes[n_train_constraint_examples:]
  val_constraint_batch = (constraint_obs_t[:, val_constraint_example_idxes],
                          constraint_act_t[:, val_constraint_example_idxes],
                          constraint_act_t_feats[:, val_constraint_example_idxes],
                          constraint_rew_t[:, val_constraint_example_idxes])

In [ ]:
def sample_constraint_batch(size):
  global n_iters_since_prev_constraint_sample
  if n_iters_since_prev_constraint_sample % constraint_sampling_freq == 0:
    sample_constraints(size)
    n_iters_since_prev_constraint_sample = 0
  n_iters_since_prev_constraint_sample += 1

  idxes = random.sample(train_constraint_example_idxes, size)
  constraint_batch = (constraint_obs_t[:, idxes],
    constraint_act_t[:, idxes],
    constraint_act_t_feats[:, idxes],
    constraint_rew_t[:, idxes])
  return constraint_batch

In [ ]:
train_constraint_example_idxes = None
val_constraint_batch = None
constraint_obs_t = None
constraint_act_t = None
constraint_act_t_feats = None
constraint_rew_t = None
n_iters_since_prev_constraint_sample = 0

In [ ]:
with open(os.path.join(data_dir, 'constraint_samples.pkl'), 'wb') as f:
  pickle.dump((
    train_constraint_example_idxes, 
    val_constraint_batch,
    constraint_obs_t,
    constraint_act_t,
    constraint_act_t_feats,
    constraint_rew_t,
    n_iters_since_prev_constraint_sample), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(os.path.join(data_dir, 'constraint_samples.pkl'), 'rb') as f:
  (train_constraint_example_idxes, 
    val_constraint_batch,
    constraint_obs_t,
    constraint_act_t,
    constraint_act_t_feats,
    constraint_rew_t,
    n_iters_since_prev_constraint_sample) = pickle.load(f)

In [ ]:
tf.global_variables_initializer().run(session=sess)

In [ ]:
n_iters = iterations * demo_obs.shape[1] // batch_size

In [ ]:
def compute_batch_loss(demo_batch, constraint_batch, step=False, trans_only=False, t=None):
  demo_batch_obs_t, demo_batch_act_t, demo_batch_act_t_feats, demo_batch_next_obs_t = demo_batch
  constraint_batch_obs_t, constraint_batch_act_t, constraint_batch_act_t_feats, constraint_batch_rew_t = constraint_batch

  feed_dict = {
    demo_obs_t_ph: demo_batch_obs_t,
    demo_act_t_ph: demo_batch_act_t,
    demo_act_t_feats_ph: demo_batch_act_t_feats,
    demo_next_obs_t_ph: demo_batch_next_obs_t,
    demo_batch_size_ph: demo_batch_obs_t.shape[1],
    constraint_obs_t_ph: constraint_batch_obs_t,
    constraint_act_t_ph: constraint_batch_act_t,
    constraint_act_t_feats_ph: constraint_batch_act_t_feats,
    constraint_rew_t_ph: constraint_batch_rew_t,
    constraint_batch_size_ph: constraint_batch_obs_t.shape[1],
  }
  
  if trans_only:
    [loss_eval] = sess.run([trans_loss], feed_dict=feed_dict)
    update_op = trans_update_op
    d = {
    'loss': loss_eval,
      }
  else:
    [loss_eval, neg_avg_log_likelihood_eval, sq_td_err_eval, trans_loss_eval] = sess.run(
        [loss, neg_avg_log_likelihood, sq_td_err, trans_loss], feed_dict=feed_dict)
    update_op = full_update_op
    d = {
        'loss': loss_eval,
        'nll': neg_avg_log_likelihood_eval,
        'ste': sq_td_err_eval,
        'tl': trans_loss_eval
      }
  
  if step:
    sess.run(update_op, feed_dict=feed_dict)
#   else:
#     d.update(compute_int_dyn_nll())
    
  return d

## Learning Dynamics with Reward

In [ ]:
full_train_logs = {
  'loss_evals': [],
  'tl_evals': [],
  'nll_evals': [],
  'ste_evals': [],
  'val_loss_evals': [],
  'val_tl_evals': [],
  'val_nll_evals': [],
  'val_ste_evals': [],
}

In [ ]:
val_log = None
while len(full_train_logs['loss_evals']) < n_iters:
  demo_batch = sample_batch(batch_size)
  constraint_batch = sample_constraint_batch(constraint_batch_size)
  t = len(full_train_logs['loss_evals'])
  train_log = compute_batch_loss(demo_batch, constraint_batch, step=True, t=t)
  if val_log is None or len(full_train_logs['loss_evals']) % val_update_freq == 0:
    val_log = compute_batch_loss(val_demo_batch, val_constraint_batch, step=False, t=t)
  
  if len(full_train_logs['loss_evals']) % 1000 == 0:
      print('%d %d %f %f %f %f' % (
        t, n_iters, train_log['loss'], train_log['tl'], val_log['loss'], val_log['tl'])
      )
  for k, v in train_log.items():
    full_train_logs['%s_evals' % k].append(v)
  for k, v in val_log.items():
    full_train_logs['%s%s_evals' % ('val_' if k in ['loss', 'nll', 'ste', 'tl'] else '', k)].append(v)
  if len(full_train_logs['loss_evals']) % 10000 == 0:
    for k in ['val_nll_evals', 'val_ste_evals']:
      plt.xlabel('Iterations')
      plt.ylabel(k.split('_')[1])
      plt.plot(full_train_logs[k])
      plt.show()

In [ ]:
for k in ['val_nll_evals', 'val_ste_evals']:
  plt.xlabel('Iterations')
  plt.ylabel(k.split('_')[1])
  plt.plot(full_train_logs[k])
  plt.show()

In [ ]:
plt.xlabel('Iterations')
plt.ylabel('Negative Log-Likelihood')
plt.plot(train_logs['int_dyn_nll_evals'], color='orange')
# plt.axhline(y=-np.log(1/n_ims), linestyle='--', color='gray', label='Uniform')
plt.ylim([-0.05, None])
plt.legend(loc='best')
plt.show()

In [ ]:
with open(os.path.join(data_dir, 'qt_results.pkl'), 'wb') as f:
  pickle.dump(full_train_logs, f)

## Learning Dynamics Without Reward

In [ ]:
train_constraint_example_idxes = None
val_constraint_batch = None
constraint_obs_t = None
constraint_act_t = None
constraint_act_t_feats = None
n_iters_since_prev_constraint_sample = 0

tf.global_variables_initializer().run(session=sess)

n_iters = iterations * demo_obs.shape[1] // batch_size

trans_train_logs = {
  'loss_evals': [],
  'nll_evals': [],
  'ste_evals': [],
  'val_loss_evals': [],
  'val_nll_evals': [],
  'val_ste_evals': [],
  'int_dyn_err_evals': []
}
val_log = None

In [ ]:
while len(trans_train_logs['loss_evals']) < n_iters:
  demo_batch = sample_batch(batch_size)
  constraint_batch = sample_constraint_batch(constraint_batch_size)
  
  t = len(trans_train_logs['loss_evals'])
  train_log = compute_batch_loss(demo_batch, constraint_batch, trans_only=True, step=True, t=t)
  if val_log is None or len(trans_train_logs['loss_evals']) % val_update_freq == 0:
    val_log = compute_batch_loss(val_demo_batch, val_constraint_batch, trans_only=True, step=False, t=t)
  
  if len(trans_train_logs['loss_evals']) % 1000 == 0:
      print('%d %d %f %f' % (
        t, n_iters, train_log['loss'], val_log['loss'],)
      )

  trans_train_logs['loss_evals'].append(train_log['loss'])
  trans_train_logs['val_loss_evals'].append(val_log['loss'])
  if len(trans_train_logs['loss_evals']) % 10000 == 0:
    for k in ['loss_evals', 'val_loss_evals']:
      plt.xlabel('Iterations')
      plt.ylabel(k.split('_')[1])
      plt.plot(trans_train_logs[k])
      plt.show()